# Gramener Case Study

In [138]:
import numpy as np
import pandas as pd
pd.options.display.float_format = '{:.3f}'.format
pd.set_option('display.max_colwidth', -1)
pd.options.display.max_columns = 20

#### External contants

In [139]:
loan_file_path="./loan/loan.csv"
data_dict_file_path = "./loan/Data_Dictionary.xlsx"

## Check Point 1:  Data Cleaning

### - Fetching ```loan``` data from given file

In [140]:
loan = pd.read_csv(loan_file_path, encoding="utf8")
loan.head()

C:\Users\Admin\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2698: DtypeWarning: Columns (47) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,...,num_tl_90g_dpd_24m,num_tl_op_past_12m,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit
0,1077501,1296599,5000,5000,4975.000,36 months,10.65%,162.870,B,B2,...,nan,nan,nan,nan,0.000,0.000,nan,nan,nan,nan
1,1077430,1314167,2500,2500,2500.000,60 months,15.27%,59.830,C,C4,...,nan,nan,nan,nan,0.000,0.000,nan,nan,nan,nan
2,1077175,1313524,2400,2400,2400.000,36 months,15.96%,84.330,C,C5,...,nan,nan,nan,nan,0.000,0.000,nan,nan,nan,nan
3,1076863,1277178,10000,10000,10000.000,36 months,13.49%,339.310,C,C1,...,nan,nan,nan,nan,0.000,0.000,nan,nan,nan,nan
4,1075358,1311748,3000,3000,3000.000,60 months,12.69%,67.790,B,B5,...,nan,nan,nan,nan,0.000,0.000,nan,nan,nan,nan


#### Finding number of rows and columns in the dataset

In [71]:
print('The loan dataset set has %d rows and %d columns' % (loan.shape[0], loan.shape[1]))

The loan dataset set has 39717 rows and 111 columns


#### Removing columns which have all values as ```NaN```

In [72]:
nan_count_df = loan.isnull().sum().to_frame('nan_count')
num_col_with_all_nan = nan_count_df.loc[nan_count_df['nan_count']  == loan.shape[0], :].shape[0]
print('Number of columns with all NaN values are %d.' % (num_col_with_all_nan))

Number of columns with all NaN values are 54.


In [144]:
# Removing columns which have all values as NaN
loans_frame = loan.dropna(axis=1, how="all")
print('The loan dataset set has now %d rows and %d columns' % (loans_frame.shape[0], loans_frame.shape[1]))

The loan dataset set has now 39717 rows and 57 columns


### - Fetching ```Data Dictionary```  from given file

In [145]:
data_dict = pd.read_excel(data_dict_file_path, sheet_name="LoanStats")
data_dict.head()

,LoanStatNew,Description
0,acc_now_delinq,The number of accounts on which the borrower is now delinquent.
1,acc_open_past_24mths,Number of trades opened in past 24 months.
2,addr_state,The state provided by the borrower in the loan application
3,all_util,Balance to credit limit on all trades
4,annual_inc,The self-reported annual income provided by the borrower during registration.


#### Initial List of columns for consideration 

In [147]:
filtered_data_dict = data_dict.loc[data_dict['LoanStatNew'].isin(loans_frame.columns), :]

### Considering those loan entries which are  either `Fully Paid` or `Charged Off`  to discover any pattern in them
The reason for not selecting `Current` loans because we don't if they will be fully paid or default in future. After creating a model we can predict if `Current` loans will default or not.

In [150]:
loans_frame = loans_frame.loc[loans_frame['loan_status'].isin(['Fully Paid', 'Charged Off']), :]
loans_frame.head()

,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,...,next_pymnt_d,last_credit_pull_d,collections_12_mths_ex_med,policy_code,application_type,acc_now_delinq,chargeoff_within_12_mths,delinq_amnt,pub_rec_bankruptcies,tax_liens
0,1077501,1296599,5000,5000,4975.000,36 months,10.65%,162.870,B,B2,...,NaN,May-16,0.000,1,INDIVIDUAL,0,0.000,0,0.000,0.000
1,1077430,1314167,2500,2500,2500.000,60 months,15.27%,59.830,C,C4,...,NaN,Sep-13,0.000,1,INDIVIDUAL,0,0.000,0,0.000,0.000
2,1077175,1313524,2400,2400,2400.000,36 months,15.96%,84.330,C,C5,...,NaN,May-16,0.000,1,INDIVIDUAL,0,0.000,0,0.000,0.000
3,1076863,1277178,10000,10000,10000.000,36 months,13.49%,339.310,C,C1,...,NaN,Apr-16,0.000,1,INDIVIDUAL,0,0.000,0,0.000,0.000
5,1075269,1311441,5000,5000,5000.000,36 months,7.90%,156.460,A,A4,...,NaN,Jan-16,0.000,1,INDIVIDUAL,0,0.000,0,0.000,0.000


In [151]:
# Shape of the 'Charged Off' records i.e. number of records and columns
loans_frame.shape

(38577, 57)

In [152]:
clean_loan_df = loans_frame.isnull().sum().sort_index().to_frame('nan_count').reset_index()

In [153]:
# Creating a dataframe by merging the data dictional the clean_loan dataframe to get a better understanding
column_desc = pd.merge(filtered_data_dict, clean_loan_df, how="inner", left_on="LoanStatNew", right_on="index")
column_desc.loc[:,['LoanStatNew','Description','nan_count']]

,LoanStatNew,Description,nan_count
0,acc_now_delinq,The number of accounts on which the borrower is now delinquent.,0
1,addr_state,The state provided by the borrower in the loan application,0
2,annual_inc,The self-reported annual income provided by the borrower during registration.,0
3,application_type,Indicates whether the loan is an individual application or a joint application with two co-borrowers,0
4,chargeoff_within_12_mths,Number of charge-offs within 12 months,56
5,collection_recovery_fee,post charge off collection fee,0
6,collections_12_mths_ex_med,Number of collections in 12 months excluding medical collections,56
7,delinq_2yrs,The number of 30+ days past-due incidences of delinquency in the borrower's credit file for the past 2 years,0
8,delinq_amnt,The past-due amount owed for the accounts on which the borrower is now delinquent.,0
9,desc,Loan description provided by the borrower,12526


In [154]:
column_desc.loc[column_desc['nan_count'] > 0,['LoanStatNew','Description','nan_count']]

,LoanStatNew,Description,nan_count
4,chargeoff_within_12_mths,Number of charge-offs within 12 months,56
6,collections_12_mths_ex_med,Number of collections in 12 months excluding medical collections,56
9,desc,Loan description provided by the borrower,12526
13,emp_title,The job title supplied by the Borrower when applying for the loan.*,2385
24,last_credit_pull_d,The most recent month LC pulled credit for this loan,2
26,last_pymnt_d,Last month payment was received,71
30,mths_since_last_delinq,The number of months since the borrower's last delinquency.,24905
31,mths_since_last_record,The number of months since the last public record.,35837
32,next_pymnt_d,Next scheduled payment date,38577
38,pub_rec_bankruptcies,Number of public record bankruptcies,697


##### Removing columns  
- ```mths_since_last_delinq```, ```mths_since_last_record```, ```next_pymnt_d``` as the contain more 50% values as NaN
- ```emp_title```, ```desc```, ```title```, ```url``` are irrelevant for EDA

In [155]:
#### Removing columns  mths_since_last_delinq, mths_since_last_record, next_pymnt_d, emp_title, desc
columns_to_remove = ['mths_since_last_delinq', 'mths_since_last_record', 'next_pymnt_d', 'emp_title', 'desc', 'url', 'title']
columns_under_process = column_desc.loc[~column_desc['LoanStatNew'].isin(columns_to_remove), 'LoanStatNew'].values
loans_frame = loans_frame.loc[:,columns_under_process]
loans_frame.head()

,acc_now_delinq,addr_state,annual_inc,application_type,chargeoff_within_12_mths,collection_recovery_fee,collections_12_mths_ex_med,delinq_2yrs,delinq_amnt,dti,...,tax_liens,term,total_acc,total_pymnt,total_pymnt_inv,total_rec_int,total_rec_late_fee,total_rec_prncp,verification_status,zip_code
0,0,AZ,24000.000,INDIVIDUAL,0.000,0.000,0.000,0,0,27.650,...,0.000,36 months,9,5863.155,5833.840,863.160,0.000,5000.000,Verified,860xx
1,0,GA,30000.000,INDIVIDUAL,0.000,1.110,0.000,0,0,1.000,...,0.000,60 months,4,1008.710,1008.710,435.170,0.000,456.460,Source Verified,309xx
2,0,IL,12252.000,INDIVIDUAL,0.000,0.000,0.000,0,0,8.720,...,0.000,36 months,10,3005.667,3005.670,605.670,0.000,2400.000,Not Verified,606xx
3,0,CA,49200.000,INDIVIDUAL,0.000,0.000,0.000,0,0,20.000,...,0.000,36 months,37,12231.890,12231.890,2214.920,16.970,10000.000,Source Verified,917xx
5,0,AZ,36000.000,INDIVIDUAL,0.000,0.000,0.000,0,0,11.200,...,0.000,36 months,12,5632.210,5632.210,632.210,0.000,5000.000,Source Verified,852xx


In [156]:
loans_frame.shape

(38577, 50)

In [157]:
print("Total number of unique member in the data set is %d" % loans_frame['member_id'].nunique())

Total number of unique member in the data set is 38577


In [158]:
pd.options.display.max_columns = 100 # To see all columns
loans_frame.describe()

,acc_now_delinq,annual_inc,chargeoff_within_12_mths,collection_recovery_fee,collections_12_mths_ex_med,delinq_2yrs,delinq_amnt,dti,funded_amnt,funded_amnt_inv,id,inq_last_6mths,installment,last_pymnt_amnt,loan_amnt,member_id,open_acc,out_prncp,out_prncp_inv,policy_code,pub_rec,pub_rec_bankruptcies,recoveries,revol_bal,tax_liens,total_acc,total_pymnt,total_pymnt_inv,total_rec_int,total_rec_late_fee,total_rec_prncp
count,38577.000,38577.000,38521.000,38577.000,38521.000,38577.000,38577.000,38577.000,38577.000,38577.000,38577.000,38577.000,38577.000,38577.000,38577.000,38577.000,38577.000,38577.000,38577.000,38577.000,38577.000,37880.000,38577.000,38577.000,38538.000,38577.000,38577.000,38577.000,38577.000,38577.000,38577.000
mean,0.000,68777.974,0.000,12.773,0.000,0.147,0.000,13.273,10784.059,10222.481,676378.707,0.872,322.466,2746.243,11047.025,842284.337,9.275,0.000,0.000,1.000,0.055,0.043,98.036,13289.490,0.000,22.053,11866.970,11271.196,2119.243,1.369,9648.323
std,0.000,64218.682,0.000,150.837,0.000,0.492,0.000,6.673,7090.306,7022.721,209263.898,1.072,208.639,4494.648,7348.442,264451.929,4.402,0.000,0.000,0.000,0.238,0.205,698.650,15866.492,0.000,11.426,8809.856,8700.846,2389.500,7.316,7009.381
min,0.000,4000.000,0.000,0.000,0.000,0.000,0.000,0.000,500.000,0.000,54734.000,0.000,15.690,0.000,500.000,70699.000,2.000,0.000,0.000,1.000,0.000,0.000,0.000,0.000,0.000,2.000,0.000,0.000,0.000,0.000,0.000
25%,0.000,40000.000,0.000,0.000,0.000,0.000,0.000,8.130,5200.000,5000.000,512033.000,0.000,165.740,217.420,5300.000,661131.000,6.000,0.000,0.000,1.000,0.000,0.000,0.000,3650.000,0.000,13.000,5513.497,5019.590,643.990,0.000,4500.000
50%,0.000,58868.000,0.000,0.000,0.000,0.000,0.000,13.370,9550.000,8733.440,656423.000,1.000,277.860,568.260,9600.000,839292.000,9.000,0.000,0.000,1.000,0.000,0.000,0.000,8762.000,0.000,20.000,9674.048,9066.660,1300.450,0.000,8000.000
75%,0.000,82000.000,0.000,0.000,0.000,0.000,0.000,18.560,15000.000,14000.000,829146.000,1.000,425.550,3447.250,15000.000,1037336.000,12.000,0.000,0.000,1.000,0.000,0.000,0.000,16912.000,0.000,29.000,16136.952,15329.780,2671.980,0.000,13175.000
max,0.000,6000000.000,0.000,7002.190,0.000,11.000,0.000,29.990,35000.000,35000.000,1077501.000,8.000,1305.190,36115.200,35000.000,1314167.000,44.000,0.000,0.000,1.000,4.000,2.000,29623.350,149588.000,0.000,90.000,58563.680,58563.680,23563.680,180.200,35000.020


#### From the above descriptive statistics we can remove the following columns as their filled with same value of very less distribution
```acc_now_delinq```, ```chargeoff_within_12_mths```, ```collections_12_mths_ex_med```, ```delinq_amnt```, ```out_prncp```, ```out_prncp_inv```, ```policy_code```, ```pub_rec```, ```pub_rec_bankruptcies```, ```tax_liens```, ```delinq_2yrs```, ```inq_last_6mths```

In [159]:
columns_to_remove.extend(['acc_now_delinq','chargeoff_within_12_mths',\
                          'collections_12_mths_ex_med','delinq_2yrs','delinq_amnt',\
                          'out_prncp','out_prncp_inv','policy_code','pub_rec','pub_rec_bankruptcies',\
                         'tax_liens', 'inq_last_6mths'])
columns_to_remove

['mths_since_last_delinq',
 'mths_since_last_record',
 'next_pymnt_d',
 'emp_title',
 'desc',
 'url',
 'title',
 'acc_now_delinq',
 'chargeoff_within_12_mths',
 'collections_12_mths_ex_med',
 'delinq_2yrs',
 'delinq_amnt',
 'out_prncp',
 'out_prncp_inv',
 'policy_code',
 'pub_rec',
 'pub_rec_bankruptcies',
 'tax_liens',
 'inq_last_6mths']

In [160]:
columns_under_process = column_desc.loc[~column_desc['LoanStatNew'].isin(columns_to_remove), 'LoanStatNew'].values
loans_frame = loans_frame.loc[:,columns_under_process]
loans_frame.head()

,addr_state,annual_inc,application_type,collection_recovery_fee,dti,earliest_cr_line,emp_length,funded_amnt,funded_amnt_inv,grade,home_ownership,id,initial_list_status,installment,int_rate,issue_d,last_credit_pull_d,last_pymnt_amnt,last_pymnt_d,loan_amnt,loan_status,member_id,open_acc,purpose,pymnt_plan,recoveries,revol_bal,revol_util,sub_grade,term,total_acc,total_pymnt,total_pymnt_inv,total_rec_int,total_rec_late_fee,total_rec_prncp,verification_status,zip_code
0,AZ,24000.000,INDIVIDUAL,0.000,27.650,Jan-85,10+ years,5000,4975.000,B,RENT,1077501,f,162.870,10.65%,Dec-11,May-16,171.620,Jan-15,5000,Fully Paid,1296599,3,credit_card,n,0.000,13648,83.70%,B2,36 months,9,5863.155,5833.840,863.160,0.000,5000.000,Verified,860xx
1,GA,30000.000,INDIVIDUAL,1.110,1.000,Apr-99,< 1 year,2500,2500.000,C,RENT,1077430,f,59.830,15.27%,Dec-11,Sep-13,119.660,Apr-13,2500,Charged Off,1314167,3,car,n,117.080,1687,9.40%,C4,60 months,4,1008.710,1008.710,435.170,0.000,456.460,Source Verified,309xx
2,IL,12252.000,INDIVIDUAL,0.000,8.720,Nov-01,10+ years,2400,2400.000,C,RENT,1077175,f,84.330,15.96%,Dec-11,May-16,649.910,Jun-14,2400,Fully Paid,1313524,2,small_business,n,0.000,2956,98.50%,C5,36 months,10,3005.667,3005.670,605.670,0.000,2400.000,Not Verified,606xx
3,CA,49200.000,INDIVIDUAL,0.000,20.000,Feb-96,10+ years,10000,10000.000,C,RENT,1076863,f,339.310,13.49%,Dec-11,Apr-16,357.480,Jan-15,10000,Fully Paid,1277178,10,other,n,0.000,5598,21%,C1,36 months,37,12231.890,12231.890,2214.920,16.970,10000.000,Source Verified,917xx
5,AZ,36000.000,INDIVIDUAL,0.000,11.200,Nov-04,3 years,5000,5000.000,A,RENT,1075269,f,156.460,7.90%,Dec-11,Jan-16,161.030,Jan-15,5000,Fully Paid,1311441,9,wedding,n,0.000,7963,28.30%,A4,36 months,12,5632.210,5632.210,632.210,0.000,5000.000,Source Verified,852xx


In [161]:
column_desc.loc[~column_desc['LoanStatNew'].isin(columns_to_remove), ['LoanStatNew', 'Description']]

,LoanStatNew,Description
1,addr_state,The state provided by the borrower in the loan application
2,annual_inc,The self-reported annual income provided by the borrower during registration.
3,application_type,Indicates whether the loan is an individual application or a joint application with two co-borrowers
5,collection_recovery_fee,post charge off collection fee
10,dti,"A ratio calculated using the borrower’s total monthly debt payments on the total debt obligations, excluding mortgage and the requested LC loan, divided by the borrower’s self-reported monthly income."
11,earliest_cr_line,The month the borrower's earliest reported credit line was opened
12,emp_length,Employment length in years. Possible values are between 0 and 10 where 0 means less than one year and 10 means ten or more years.
14,funded_amnt,The total amount committed to that loan at that point in time.
15,funded_amnt_inv,The total amount committed by investors for that loan at that point in time.
16,grade,LC assigned loan grade


### Check Point 2: Univariate Analysis

From the above results we can now chalk out catergorical columsn in the ```loans_frame```
- ```unordered categorical variables``` 
==> addr_state, application_type, home_ownership, initial_list_status, purpose, pymnt_plan, sub_grade, verification_status, zip_code
- ```ordered categorical variables```
==> emp_length, grade, sub_grade, term

In [274]:
def category_univariate_analysis(column_name):
    df = loans_frame.groupby('loan_status')[column_name].value_counts(ascending=False)\
                            .unstack(level=0).reset_index()
    df['Total'] = df['Charged Off'] + df['Fully Paid']
    df['Charged_Off_Percent '] = df['Charged Off'] * 100 /df['Total']
    df['Fully_Paid_Percent'] = df['Fully Paid'] * 100 /df['Total']
    return df

In [275]:
loans_frame['verification_status'].value_counts(normalize=True)

Not Verified      0.433
Verified          0.316
Source Verified   0.251
Name: verification_status, dtype: float64

In [276]:
category_univariate_analysis('verification_status')

loan_status,verification_status,Charged Off,Fully Paid,Total,Charged_Off_Percent,Fully_Paid_Percent
0,Not Verified,2142,14552,16694,12.831,87.169
1,Source Verified,1434,8243,9677,14.819,85.181
2,Verified,2051,10155,12206,16.803,83.197


In [257]:
loans_frame['home_ownership'].value_counts(normalize=True)

RENT       0.479
MORTGAGE   0.441
OWN        0.077
OTHER      0.003
NONE       0.000
Name: home_ownership, dtype: float64

In [277]:
category_univariate_analysis('home_ownership')

loan_status,home_ownership,Charged Off,Fully Paid,Total,Charged_Off_Percent,Fully_Paid_Percent
0,MORTGAGE,2327.000,14694.000,17021.000,13.671,86.329
1,NONE,nan,3.000,nan,nan,nan
2,OTHER,18.000,80.000,98.000,18.367,81.633
3,OWN,443.000,2532.000,2975.000,14.891,85.109
4,RENT,2839.000,15641.000,18480.000,15.363,84.637


In [258]:
loans_frame['purpose'].value_counts(normalize=True)

debt_consolidation   0.468
credit_card          0.130
other                0.100
home_improvement     0.075
major_purchase       0.056
small_business       0.045
car                  0.039
wedding              0.024
medical              0.018
moving               0.015
vacation             0.010
house                0.010
educational          0.008
renewable_energy     0.003
Name: purpose, dtype: float64

In [259]:
category_univariate_analysis('purpose')

loan_status,purpose,Charged Off,Fully Paid,Total,Charged_Off_Percent,Fully_Paid_Percent
0,car,160,1339,1499,10.674,89.326
1,credit_card,542,4485,5027,10.782,89.218
2,debt_consolidation,2767,15288,18055,15.325,84.675
3,educational,56,269,325,17.231,82.769
4,home_improvement,347,2528,2875,12.070,87.930
5,house,59,308,367,16.076,83.924
6,major_purchase,222,1928,2150,10.326,89.674
7,medical,106,575,681,15.565,84.435
8,moving,92,484,576,15.972,84.028
9,other,633,3232,3865,16.378,83.622


In [284]:
loans_frame['grade'].value_counts(normalize=True)

B   0.303
A   0.260
C   0.203
D   0.132
E   0.069
F   0.025
G   0.008
Name: grade, dtype: float64

In [279]:
category_univariate_analysis('grade')

loan_status,grade,Charged Off,Fully Paid,Total,Charged_Off_Percent,Fully_Paid_Percent
0,A,602,9443,10045,5.993,94.007
1,B,1425,10250,11675,12.206,87.794
2,C,1347,6487,7834,17.194,82.806
3,D,1118,3967,5085,21.986,78.014
4,E,715,1948,2663,26.849,73.151
5,F,319,657,976,32.684,67.316
6,G,101,198,299,33.779,66.221


In [280]:
category_univariate_analysis('sub_grade')

loan_status,sub_grade,Charged Off,Fully Paid,Total,Charged_Off_Percent,Fully_Paid_Percent
0,A1,30,1109,1139,2.634,97.366
1,A2,74,1434,1508,4.907,95.093
2,A3,103,1707,1810,5.691,94.309
3,A4,178,2695,2873,6.196,93.804
4,A5,217,2498,2715,7.993,92.007
5,B1,171,1626,1797,9.516,90.484
6,B2,228,1773,2001,11.394,88.606
7,B3,341,2484,2825,12.071,87.929
8,B4,329,2108,2437,13.500,86.500
9,B5,356,2259,2615,13.614,86.386


In [285]:
loans_frame['pymnt_plan'].value_counts(normalize=True)

n   1.000
Name: pymnt_plan, dtype: float64

In [286]:
loans_frame['application_type'].value_counts(normalize=True)

INDIVIDUAL   1.000
Name: application_type, dtype: float64

In [287]:
loans_frame['emp_length'].value_counts(normalize=True)

10+ years   0.220
< 1 year    0.117
2 years     0.111
3 years     0.104
4 years     0.087
5 years     0.083
1 year      0.082
6 years     0.056
7 years     0.044
8 years     0.037
9 years     0.032
n/a         0.027
Name: emp_length, dtype: float64

In [288]:
category_univariate_analysis('emp_length')

loan_status,emp_length,Charged Off,Fully Paid,Total,Charged_Off_Percent,Fully_Paid_Percent
0,1 year,456,2713,3169,14.389,85.611
1,10+ years,1331,7157,8488,15.681,84.319
2,2 years,567,3724,4291,13.214,86.786
3,3 years,555,3457,4012,13.833,86.167
4,4 years,462,2880,3342,13.824,86.176
5,5 years,458,2736,3194,14.339,85.661
6,6 years,307,1861,2168,14.161,85.839
7,7 years,263,1448,1711,15.371,84.629
8,8 years,203,1232,1435,14.146,85.854
9,9 years,158,1068,1226,12.887,87.113


In [289]:
loans_frame['term'].value_counts(normalize=True)

 36 months   0.754
 60 months   0.246
Name: term, dtype: float64

In [290]:
category_univariate_analysis('term')

loan_status,term,Charged Off,Fully Paid,Total,Charged_Off_Percent,Fully_Paid_Percent
0,36 months,3227,25869,29096,11.091,88.909
1,60 months,2400,7081,9481,25.314,74.686


In [293]:
loans_frame['initial_list_status'].value_counts()

f    38577
Name: initial_list_status, dtype: int64

In [292]:
loans_frame['addr_state'].value_counts(normalize=True)

CA   0.180
NY   0.096
FL   0.072
TX   0.069
NJ   0.046
IL   0.038
PA   0.038
VA   0.035
GA   0.035
MA   0.034
OH   0.031
MD   0.027
AZ   0.022
WA   0.021
CO   0.020
NC   0.019
CT   0.019
MI   0.018
MO   0.017
MN   0.016
NV   0.012
SC   0.012
WI   0.011
OR   0.011
AL   0.011
LA   0.011
KY   0.008
OK   0.007
KS   0.007
UT   0.007
AR   0.006
DC   0.005
RI   0.005
NM   0.005
WV   0.004
HI   0.004
NH   0.004
DE   0.003
MT   0.002
WY   0.002
AK   0.002
SD   0.002
VT   0.001
MS   0.000
TN   0.000
IN   0.000
ID   0.000
IA   0.000
NE   0.000
ME   0.000
Name: addr_state, dtype: float64

In [294]:
category_univariate_analysis('addr_state')

loan_status,addr_state,Charged Off,Fully Paid,Total,Charged_Off_Percent,Fully_Paid_Percent
0,AK,15.000,63.000,78.000,19.231,80.769
1,AL,54.000,381.000,435.000,12.414,87.586
2,AR,27.000,208.000,235.000,11.489,88.511
3,AZ,123.000,726.000,849.000,14.488,85.512
4,CA,1125.000,5824.000,6949.000,16.189,83.811
5,CO,98.000,668.000,766.000,12.794,87.206
6,CT,94.000,632.000,726.000,12.948,87.052
7,DC,15.000,196.000,211.000,7.109,92.891
8,DE,12.000,101.000,113.000,10.619,89.381
9,FL,504.000,2277.000,2781.000,18.123,81.877


#### Quatitative variable univariate analysis

In [298]:
loans_frame.loc[loans_frame['loan_status'] == 'Charged Off','dti'].describe()

count   5627.000
mean    14.001  
std     6.586   
min     0.000   
25%     9.050   
50%     14.290  
75%     19.290  
max     29.850  
Name: dti, dtype: float64

In [299]:
loans_frame.loc[loans_frame['loan_status'] == 'Fully Paid','dti'].describe()

count   32950.000
mean    13.148   
std     6.680    
min     0.000    
25%     7.980    
50%     13.200   
75%     18.390   
max     29.990   
Name: dti, dtype: float64